In [11]:
import requests
from bs4 import BeautifulSoup
import ollama
from ollama import Client
import time
import asyncio
import aiohttp
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
def client(content):
    client = Client(
        host='http://localhost:11434',
        verify=False,  # 关闭 SSL 验证
        headers={'x-some-header': 'some-value'}
    )
    response = client.chat(model='llama3.2', messages=[
        {
        'role': 'user',
        'content': content,
        },
    ])
    return response.message.content
# 初始化翻译器和概要生成器，使用本地部署的 ollama 模型
def translator(text,target_language):
    response = client(f'英文翻译成中文：\n{text}')
    return response
def summarizer(text,max_length=150, min_length=30, do_sample=False):
    response = client(f'summarize the article\n{text}')
    return response.message.content


# 异步函数来获取新闻内容并生成概要
async def fetch_and_summarize(driver, url):
    try:
        async with driver.get(url) as response:
            if response.status != 200:
                return "未能获取新闻内容"
            news_html_content = await response.text()
            news_soup = BeautifulSoup(news_html_content, 'html.parser')
            news_article = news_soup.find('article')
            if news_article:
                news_text = news_article.get_text(separator=' ')
                # 生成新闻概要
                summary =  summarizer(news_text, max_length=150, min_length=30, do_sample=False)
                # 翻译新闻概要
                translated_summary =  translator(summary, target_language='zh')
                return translated_summary
            else:
                return "未能提取新闻内容"
    except Exception as e:
        return f"请求出错: {str(e)}"




In [12]:
# 设置 Chrome 选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式，不打开浏览器窗口
chrome_options.add_argument("--disable-gpu")
# 等待页面加载完成，可以使用显式等待或隐式等待
# 这里使用显式等待来等待特定元素出现
# 定义一个函数来等待包含特定子字符串类名的元素出现

def element_has_class(driver, class_substring):
    elements = driver.find_elements(By.XPATH, f"//*[contains(@class, '{class_substring}')]")
    return len(elements) > 0
# 指定 ChromeDriver 的路径
chrome_driver_path = 'D:/vnpy/chromedriver.exe'
def get_driver(chrome_driver_path):
    try:
        # 创建 WebDriver 对象
        service = Service(chrome_driver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)
        return driver
    except Exception as e:
        print(f"获取driver出错: {str(e)}")
        exit()
def load():
        # 抓取网页内容
    # 1. 访问网页并等待加载完成
    url = "https://www.bloomberg.com/latest?utm_source=homepage&utm_medium=web&utm_campaign=latest"
    try:
        driver=get_driver(chrome_driver_path)
        driver.get(url)
    except Exception as e:
        print(f'url 页面加载出错！\n{e}')
        raise SystemExit  # 抛出 SystemExit 异常

    # 等待新闻列表的 div 元素出现
    try:
        WebDriverWait(driver, 10).until(lambda driver: element_has_class(driver, 'LineupContentArchiveFiltered_itemContainer'))
    except Exception as e:
        print(f"页面加载出错: {e}")
        driver.quit()
        # exit() #会导致内核退出
        raise SystemExit  # 抛出 SystemExit 异常

    # 获取页面内容
    html_content = driver.page_source

    # 2. 解析网页内容
    news_div_elements = driver.find_elements(By.XPATH, f"//*[contains(@class, 'LineupContentArchiveFiltered_itemContainer')]")

    markets_elements = driver.find_elements(By.XPATH, f"//*[contains(@class, 'LineupContentArchiveFiltered_currentFilter__9qxUE')]")
    markets_element=markets_elements[0] if len(markets_elements)>0 else None
    if markets_element:
        # 等待新闻列表的 div 元素出现
        try:
            markets_element.click()
            WebDriverWait(driver, 10).until(lambda driver: element_has_class(driver, 'LineupContentArchiveFiltered_currentFilter__9qxUE'))
            news_elements = driver.find_elements(By.XPATH, f"//*[contains(@class, 'LineupContentArchiveFiltered_itemContainer')]")
            news_div_elements = news_div_elements + news_elements
        except Exception as e:
            print(f"markets页面加载出错: {str(e)}")

    if not news_div_elements:
        print("未找到新闻列表的 div 元素")
        driver.quit()
        raise SystemExit  # 抛出 SystemExit 异常
    return news_div_elements


In [13]:
# 提取新闻的标题、链接和时间
def get_news_list():    
    news_div_elements=load()
    news_list = []
    for news in news_div_elements:
        timestamp_elem = news.find_element(By.XPATH, ".//*[contains(@class, 'LineupContentArchiveFiltered_itemTimestamp')]")
        if timestamp_elem:
            timestamp = timestamp_elem.text.strip()
        else:
            timestamp = "未知时间"

        story_link_elem = news.find_element(By.XPATH, ".//*[contains(@class, 'LineupContentArchiveFiltered_storyLink') and @href]")
        if story_link_elem:
            news_title = story_link_elem.text.strip()
            news_url = story_link_elem.get_attribute('href')
            news_list.append((timestamp, news_title, news_url))
        else:
            continue
    return news_list

In [14]:
# 异步主函数
async def main(news_list):
    tasks = []
    for timestamp, news_title, news_url in news_list:
        # 翻译新闻标题
        translated_title = translator(news_title, target_language='zh')
        print(f"标题：{translated_title}")
        # 创建并添加任务
        task = fetch_and_summarize(driver, news_url)
        tasks.append(task)

    # 等待所有任务完成
    translated_summaries = await asyncio.gather(*tasks)

    # 输出结果
    for (timestamp, news_title, news_url), translated_summary in zip(news_list, translated_summaries):
        print(f"时间: {timestamp}")
        print(f"新闻标题: {translated_title}")
        print(f"新闻概要: {translated_summary}")
        print("-" * 50)

# 运行异步主函数
async def run_main():
    while True:
        news_list=get_news_list()
        await main(news_list)
        time.sleep(5*60)

# 获取当前事件循环并运行协程
# loop = asyncio.get_event_loop()
# loop.run_until_complete(run_main())
await run_main()

标题：《英国统计数据在多个方面都失败了，引发新的政策风险》

原文：

"UK Statistics Fail to Deliver on Multiple Fronts Pose New Policy Risk"

中文翻译：

《英国统计数据在多个方面都失败了，引发新的政策风险》

 UK政府的统计数据因在预测和决策方面的失误而造成了新一轮的政策风险。

英国统计局（Office for National Statistics, ONS）的数据一直被视为政策制定者的必备工具。然而，2022 年 7 月 19 日，英国统计局发布的一份报告显示，英国の人口增长速度比预期要快，这意味着政府需要重新估计其人口相关的税收和社会保险制度。这一发现引发了人们对英国统计局数据准确性以及政策制定者的依赖程度的担忧。

此外，2023 年 1 月 31 日，英国统计局发布的一份报告显示，在过去十年，英国の经济增长率低于预期。这种低增长率意味着英国政府将面临更高的财政负担，这可能会影响其在未来几年实现的经济目标。

这些失败让人们意识到，政策制定者依赖统计数据的程度很高，但也需要对这种依赖有所准备。在未来的决策过程中，英国政府应考虑多种视角和数据来源，以确保其政策决策更为可靠。
标题：以下是英文原文的中文翻译：

越南 VinFast 新建电动汽车工厂预计于7月开始运营
标题：原文：

"Biosecure Act Moves Out of Key Defense Bill in Win for Chinese Biotech"

中文翻译：

“生物安全法案”出现在关键国防法案中取得了胜利，这是中国生物技术的好处。

或译：“生物安全法案在关键国防法案中脱颖而出，成为了中国生物技术取得的重要胜利。”

这篇文章的主要内容是关于美国的一项法律，即“生物安全法案”，该法案出现在一个关键的国防法案中。该法案通过了，并且对中国生物技术有一定影响。

中文翻译后，得到以下结果：

该法案的确是在国防法案中取得了胜利，但是具体内容和影响需要进一步详解。

如果需要，我可以提供更多关于这项法案以及其对中国生物技术的影响的信息。
标题：“叙利亚总统阿萨德政府被反叛军夺取”

根据近期的报道，叙利亚政府的控制权正在逐渐丧失，反叛军在叙利亚内战中取得了重要的胜利。 2011年起，叙利亚内战爆发，反叛军

SystemExit: 